In [ ]:
%%cmd
python -m pip install flatten_json
python -m pip install pandas

In [ ]:
import pandas as pd
import json
from collections import defaultdict
from flatten_json import flatten

In [ ]:
files = [
    '../../a_ray_tracing_inline_deep.json',
    '../../b_ray_tracing_inline_deep.json',
    '../../a_ray_tracing_pipeline_deep.json',
    '../../b_ray_tracing_pipeline_deep.json',
]

In [ ]:
def aggregate_loops_passes(json):
    results_per_frame = []
    num_loops = len(json)
    for loop_results in json:
        for frame_index, frame_results in enumerate(loop_results["per_frame_results"]):
            if frame_index >= len(results_per_frame):
                results_per_frame.append(defaultdict(int))
            results_per_frame[frame_index]['sequence_time_ns'] = frame_results['sequence_time_ns']
            for command_buffer_timings in frame_results[
                "command_buffer_timings"
            ].values():
                for scope_name, scope_timings in command_buffer_timings[
                    "scope_timings"
                ].items():
                    for scope_timing in scope_timings:
                        results_per_frame[frame_index][scope_name] += (
                            scope_timing["end"] - scope_timing["start"]
                        ) / num_loops / 1_000_000 # in ms
            for metric_name, metric in frame_results["metrics"].items():
                # TODO: Flatten this in rust to fan_speed_rpm
                if metric_name == "fan_speed":
                    value = metric["Percent"] if "Percent" in metric else metric["Rpm"]
                    results_per_frame[frame_index]["fan_speed_rpm"] += (
                        value / num_loops
                    )
                # Filter out unavailable data and the timestamp
                elif metric is not None and metric_name != "timestamp":
                    results_per_frame[frame_index][metric_name] += metric / num_loops
    # TODO: Aggregate CPU timings
    return pd.DataFrame([flatten(x) for x in results_per_frame])

results = {}

# Load all files into one large dataframe
for path in files:
    with open(path, "r") as json_file:
        # We aggregate passes within each frame, so we get one number per pass per frame per input file
        json_data = aggregate_loops_passes(json.load(json_file))
    results[path] = json_data
# Concat into input file + sequence time per row, metric per column
full_dataset = pd.concat(results)
full_dataset

In [ ]:
# Print all possible metrics
full_dataset.columns.tolist()

In [ ]:
relevant_metrics = [
    'reflection-hits-shading',
    'water-compositing',
    'blur',
    'diffuse-spatial-filter',
    'spd',
    'sun-direct-lighting',
    'reflection-ray-tracing-inline',
    'trace-diffuse-nee-rays',
    'render pass',
    'shadow-ray-tracing-pipeline',
    'compositing',
    'build-gbuffers_0',
    'scale-raster',
    'Batch refit bottom level',
    'clock_speed_in_mhz',
    'board_power_usage_in_w',
    'vram_usage_in_mb',
    'edge_temperature_in_c'
]
# We want all relevant metrics with the sequence time, to properly plot on the x axis
relevant_metrics_with_time = relevant_metrics + ["sequence_time_ns"]
metrics = full_dataset[relevant_metrics_with_time]

# Reshape into sequence time + metric type per row, input file per column
metrics = metrics.reset_index().set_index(['sequence_time_ns', 'level_0']).drop('level_1', axis=1)
metrics = metrics.stack().unstack(1).reset_index()

# From ns to s
metrics['sequence_time_s'] = metrics['sequence_time_ns'] / 1_000_000_000
metrics = metrics.drop('sequence_time_ns', axis=1)
metrics

In [ ]:
for graph_name in metrics['level_1'].unique():
    # Grab the metric we want to plot
    selected_metric = metrics[metrics['level_1'] == graph_name]
    selected_metric = selected_metric.drop('level_1', axis=1)

    # Filter outliers out of view
    max_mean = selected_metric[files].mean().mean()
    max_mean = 0 if pd.isna(max_mean) else max_mean
    max_std = selected_metric[files].std(axis=1).max()
    max_std = selected_metric[files].max() / 3.0 if pd.isna(max_std) else max_std

    # Plot results 
    selected_metric.infer_objects(copy=False).interpolate(method='linear').plot(
        x='sequence_time_s', 
        ylabel='shader execution time in ms',
        xlabel='benchmark timeline in seconds', 
        ylim= (max(0, max_mean - max_std * 3), max_mean + max_std * 3),
        figsize=(20,10), 
        colormap='Dark2', 
        grid=True, 
        legend=True,
        title=graph_name
    )